## Imports

In [ ]:
import os
from glob import glob
from tqdm import tqdm

import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from scipy.integrate import simpson
from scipy.optimize import curve_fit
from scipy.signal import argrelextrema
from scipy.stats import spearmanr, median_abs_deviation

from astropy.io import fits
from astropy.convolution import convolve, Gaussian1DKernel
from astropy.stats import sigma_clipped_stats

%matplotlib inline

In [ ]:
plt.rcParams.update({'font.size': 20})

## Load Lines and Data

In [ ]:
data_path = '../data'
lines_vac, lines_air = np.loadtxt(f'{data_path}/hydroxyl_lines_rousselot_2000.txt').T

In [ ]:
results_path = '../results'
df = pd.read_csv(f'{results_path}/lines_norm_gdl_fit.csv', index_col=0)
columns = df.columns

In [ ]:
col_sub = []
for column in columns[9:28]:
    if 'sigma1_cov' in column or 'sigma2_cov' in column:
        col_sub.append(column[9:])
    else:
        col_sub.append(column.split('_')[1])

In [ ]:
col_sub = np.array(col_sub)

In [ ]:
col_sub

In [ ]:
time_centered = df['spec_mjd'] - int(df['spec_mjd'].min())

In [ ]:
mask1_1 = (time_centered > 504) & (time_centered < 504.5)
mask1_2 = (time_centered > 504.58) & (time_centered < 505.515)
mask1_3 = (time_centered > 505.58) & (time_centered < 506.578)
mask1_4 = (time_centered > 506.61) & (time_centered < 507)
mask1 = mask1_1 | mask1_2 | mask1_3 | mask1_4
mask2 = (time_centered > 543) & (time_centered < 547)

In [ ]:
def append_all_lines(df, column, lines_vac, time_centered, use_mask, mask):
    
    all_time = []
    all_column = []
    all_lines = []
    
    for line in lines_vac:
        line_column = str(line) + '_' + column
        if use_mask:
            all_time.append(time_centered.values[mask])
            all_column.append(df[line_column].values[mask])
            all_lines.append(np.ones(df.shape[0])[mask]*line)
        else:
            all_time.append(time_centered.values)
            all_column.append(df[line_column].values)
            all_lines.append(np.ones(df.shape[0])*line)
            
    all_time = np.concatenate(all_time)
    all_column = np.concatenate(all_column)
    all_lines = np.concatenate(all_lines)
    
    return all_time, all_column, all_lines/10000

## Local Median

In [ ]:
x, y, c = append_all_lines(df, col_sub[0], lines_vac, time_centered, use_mask=0, mask=0)
x1, y1, c1 = append_all_lines(df, col_sub[0], lines_vac, time_centered, use_mask=1, mask=mask1)
x2, y2, c2 = append_all_lines(df, col_sub[0], lines_vac, time_centered, use_mask=1, mask=mask2)

In [ ]:
fs = 45
y_min = -10**2
y_max = 10**2
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: Local Median', fontsize=fs)
plt.scatter(x, y, c=c, alpha=0.5, s=10, cmap='plasma')
plt.vlines(505.5, y_min, y_max, color='C5', linewidth=5, label='Event 1', ls='--')
plt.vlines(545, y_min, y_max, color='C7', linewidth=5, label='Event 2', ls='--')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Relative Flux', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.yscale('symlog')
plt.ylim(y_min, y_max)
plt.legend(fontsize=fs)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

In [ ]:
fs = 45
y_min = -10**2
y_max = 10**2
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: Local Median (Event 1)', fontsize=fs)
plt.scatter(x1, y1, c=c1, alpha=0.5, s=1, cmap='plasma')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Relative Flux', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.yscale('symlog')
plt.ylim(y_min, y_max)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

In [ ]:
fs = 45
y_min = -10**2
y_max = 10**2
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: Local Median (Event 2)', fontsize=fs)
plt.scatter(x2, y2, c=c2, alpha=0.5, s=1, cmap='plasma')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Relative Flux', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.yscale('symlog')
plt.ylim(y_min, y_max)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

## MAD

In [ ]:
x, y, c = append_all_lines(df, col_sub[1], lines_vac, time_centered, use_mask=0, mask=0)
x1, y1, c1 = append_all_lines(df, col_sub[1], lines_vac, time_centered, use_mask=1, mask=mask1)
x2, y2, c2 = append_all_lines(df, col_sub[1], lines_vac, time_centered, use_mask=1, mask=mask2)

In [ ]:
fs = 45
y_min = 10**-4
y_max = 10**2
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: Local Median Absolute Deviation', fontsize=fs)
plt.scatter(x, y, c=c, alpha=0.5, s=10, cmap='plasma')
plt.vlines(505.5, y_min, y_max, color='C5', linewidth=5, label='Event 1', ls='--')
plt.vlines(545, y_min, y_max, color='C7', linewidth=5, label='Event 2', ls='--')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Relative Flux', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.yscale('log')
plt.ylim(y_min, y_max)
plt.legend(fontsize=fs)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

In [ ]:
fs = 45
y_min = 10**-4
y_max = 10**2
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: Local Median Absolute Deviation (Event 1)', fontsize=fs)
plt.scatter(x1, y1, c=c1, alpha=0.5, s=1, cmap='plasma')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Relative Flux', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.yscale('log')
plt.ylim(y_min, y_max)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

In [ ]:
fs = 45
y_min = 10**-4
y_max = 10**2
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: Local Median Absolute Deviation (Event 2)', fontsize=fs)
plt.scatter(x2, y2, c=c2, alpha=0.5, s=1, cmap='plasma')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Relative Flux', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.yscale('log')
plt.ylim(y_min, y_max)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

## Sum

In [ ]:
x, y, c = append_all_lines(df, col_sub[2], lines_vac, time_centered, use_mask=0, mask=0)
x1, y1, c1 = append_all_lines(df, col_sub[2], lines_vac, time_centered, use_mask=1, mask=mask1)
x2, y2, c2 = append_all_lines(df, col_sub[2], lines_vac, time_centered, use_mask=1, mask=mask2)

In [ ]:
fs = 45
y_min = -10**3
y_max = 10**3
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: Total Flux Contribution', fontsize=fs)
plt.scatter(x, y, c=c, alpha=0.5, s=10, cmap='plasma')
plt.vlines(505.5, y_min, y_max, color='C5', linewidth=5, label='Event 1', ls='--')
plt.vlines(545, y_min, y_max, color='C7', linewidth=5, label='Event 2', ls='--')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Relative Flux', fontsize=fs)
plt.tick_params(labelsize=40)
plt.yscale('symlog')
plt.ylim(y_min, y_max)
plt.legend(fontsize=fs)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

In [ ]:
fs = 45
y_min = -10**3
y_max = 10**3
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: Total Flux Contribution (Event 1)', fontsize=fs)
plt.scatter(x1, y1, c=c1, alpha=0.5, s=1, cmap='plasma')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Relative Flux', fontsize=fs)
plt.tick_params(labelsize=40)
plt.yscale('symlog')
plt.ylim(y_min, y_max)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

In [ ]:
fs = 45
y_min = -10**3
y_max = 10**3
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: Total Flux Contribution (Event 2)', fontsize=fs)
plt.scatter(x2, y2, c=c2, alpha=0.5, s=1, cmap='plasma')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Relative Flux', fontsize=fs)
plt.tick_params(labelsize=40)
plt.yscale('symlog')
plt.ylim(y_min, y_max)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

## Amp

In [ ]:
x, y, c = append_all_lines(df, col_sub[3], lines_vac, time_centered, use_mask=0, mask=0)
x1, y1, c1 = append_all_lines(df, col_sub[3], lines_vac, time_centered, use_mask=1, mask=mask1)
x2, y2, c2 = append_all_lines(df, col_sub[3], lines_vac, time_centered, use_mask=1, mask=mask2)

In [ ]:
x_, y_, c_ = append_all_lines(df, col_sub[14], lines_vac, time_centered, use_mask=0, mask=0)
x1_, y1_, c1_ = append_all_lines(df, col_sub[14], lines_vac, time_centered, use_mask=1, mask=mask1)
x2_, y2_, c2_ = append_all_lines(df, col_sub[14], lines_vac, time_centered, use_mask=1, mask=mask2)

In [ ]:
x = np.concatenate((x, x_))
y = np.concatenate((y, y_))
c = np.concatenate((c, c_))

In [ ]:
fs = 45
y_min = -10**3
y_max = 10**3
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: $A_1$ and $A_2$', fontsize=fs)
plt.scatter(x, y, c=c, alpha=0.25, s=10, cmap='plasma')
plt.vlines(505.5, y_min, y_max, color='C5', linewidth=5, label='Event 1', ls='--')
plt.vlines(545, y_min, y_max, color='C7', linewidth=5, label='Event 2', ls='--')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Relative Flux $\cdot$ $\mathring{A}$', fontsize=fs)
plt.tick_params(labelsize=40)
plt.yscale('symlog')
plt.ylim(y_min, y_max)
plt.legend(fontsize=fs)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

In [ ]:
x1 = np.concatenate((x1, x1_))
y1 = np.concatenate((y1, y1_))
c1 = np.concatenate((c1, c1_))

In [ ]:
fs = 45
y_min = -10**3
y_max = 10**3
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: $A_1$ and $A_2$ (Event 1)', fontsize=fs)
plt.scatter(x1, y1, c=c1, alpha=0.25, s=1, cmap='plasma')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Relative Flux $\cdot$ $\mathring{A}$', fontsize=fs)
plt.tick_params(labelsize=40)
plt.yscale('symlog')
plt.ylim(y_min, y_max)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)
plt.show()

In [ ]:
x2 = np.concatenate((x2, x2_))
y2 = np.concatenate((y2, y2_))
c2 = np.concatenate((c2, c2_))

In [ ]:
fs = 45
y_min = -10**3
y_max = 10**3
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: $A_1$ and $A_2$ (Event 2)', fontsize=fs)
plt.scatter(x2, y2, c=c2, alpha=0.25, s=1, cmap='plasma')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Relative Flux $\cdot$ $\mathring{A}$', fontsize=fs)
plt.tick_params(labelsize=40)
plt.yscale('symlog')
plt.ylim(y_min, y_max)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

## Mu

In [ ]:
x, y, c = append_all_lines(df, col_sub[4], lines_vac, time_centered, use_mask=0, mask=0)
x1, y1, c1 = append_all_lines(df, col_sub[4], lines_vac, time_centered, use_mask=1, mask=mask1)
x2, y2, c2 = append_all_lines(df, col_sub[4], lines_vac, time_centered, use_mask=1, mask=mask2)

In [ ]:
x_, y_, c_ = append_all_lines(df, col_sub[15], lines_vac, time_centered, use_mask=0, mask=0)
x1_, y1_, c1_ = append_all_lines(df, col_sub[15], lines_vac, time_centered, use_mask=1, mask=mask1)
x2_, y2_, c2_ = append_all_lines(df, col_sub[15], lines_vac, time_centered, use_mask=1, mask=mask2)

In [ ]:
x = np.concatenate((x, x_))
y = np.concatenate((y, y_))
c = np.concatenate((c, c_))

In [ ]:
fs = 45
y_min = 1
y_max = 2.25
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: $\mu_1$ and $\mu_2$', fontsize=fs)
plt.scatter(x, y/10000, c=c, alpha=0.25, s=10, cmap='plasma')
plt.vlines(505.5, y_min, y_max, color='C5', linewidth=5, label='Event 1', ls='--')
plt.vlines(545, y_min, y_max, color='C7', linewidth=5, label='Event 2', ls='--')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Wavelength ($\mu$m)', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.legend(fontsize=fs)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

In [ ]:
x1 = np.concatenate((x1, x1_))
y1 = np.concatenate((y1, y1_))
c1 = np.concatenate((c1, c1_))

In [ ]:
fs = 45
y_min = 1
y_max = 2.25
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: $\mu_1$ and $\mu_2$ (Event 1)', fontsize=fs)
plt.scatter(x1, y1/10000, c=c1, alpha=0.25, s=1, cmap='plasma')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Wavelength ($\mu$m)', fontsize=fs)
plt.tick_params(labelsize=fs)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

In [ ]:
x2 = np.concatenate((x2, x2_))
y2 = np.concatenate((y2, y2_))
c2 = np.concatenate((c2, c2_))

In [ ]:
fs = 45
y_min = 1
y_max = 2.25
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: $\mu_1$ and $\mu_2$ (Event 2)', fontsize=fs)
plt.scatter(x2, y2/10000, c=c2, alpha=0.25, s=1, cmap='plasma')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Wavelength ($\mu$m)', fontsize=fs)
plt.tick_params(labelsize=fs)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

## Sigma

In [ ]:
x, y, c = append_all_lines(df, col_sub[5], lines_vac, time_centered, use_mask=0, mask=0)
x1, y1, c1 = append_all_lines(df, col_sub[5], lines_vac, time_centered, use_mask=1, mask=mask1)
x2, y2, c2 = append_all_lines(df, col_sub[5], lines_vac, time_centered, use_mask=1, mask=mask2)

In [ ]:
x_, y_, c_ = append_all_lines(df, col_sub[16], lines_vac, time_centered, use_mask=0, mask=0)
x1_, y1_, c1_ = append_all_lines(df, col_sub[16], lines_vac, time_centered, use_mask=1, mask=mask1)
x2_, y2_, c2_ = append_all_lines(df, col_sub[16], lines_vac, time_centered, use_mask=1, mask=mask2)

In [ ]:
x = np.concatenate((x, x_))
y = np.concatenate((y, y_))
c = np.concatenate((c, c_))

In [ ]:
fs = 45
y_min = 10**-12
y_max = 10**-2
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: $\sigma_1$ and $\sigma_2$', fontsize=fs)
plt.scatter(x, y/10000, c=c, alpha=0.25, s=10, cmap='plasma')
plt.vlines(505.5, y_min, y_max, color='C5', linewidth=5, label='Event 1', ls='--')
plt.vlines(545, y_min, y_max, color='C7', linewidth=5, label='Event 2', ls='--')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Wavelength ($\mu$m)', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.yscale('log')
plt.ylim(y_min, y_max)
plt.legend(fontsize=fs)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

In [ ]:
x1 = np.concatenate((x1, x1_))
y1 = np.concatenate((y1, y1_))
c1 = np.concatenate((c1, c1_))

In [ ]:
fs = 45
y_min = 10**-12
y_max = 10**-2
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: $\sigma_1$ and $\sigma_2$ (Event 1)', fontsize=fs)
plt.scatter(x1, y1/10000, c=c1, alpha=0.25, s=1, cmap='plasma')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Wavelength ($\mu$m)', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.yscale('log')
plt.ylim(y_min, y_max)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

In [ ]:
x2 = np.concatenate((x2, x2_))
y2 = np.concatenate((y2, y2_))
c2 = np.concatenate((c2, c2_))

In [ ]:
fs = 45
y_min = 10**-12
y_max = 10**-2
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: $\sigma_1$ and $\sigma_2$ (Event 2)', fontsize=fs)
plt.scatter(x2, y2/10000, c=c2, alpha=0.25, s=1, cmap='plasma')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Wavelength ($\mu$m)', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.yscale('log')
plt.ylim(y_min, y_max)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

## Error

In [ ]:
x, y, c = append_all_lines(df, col_sub[7], lines_vac, time_centered, use_mask=0, mask=0)
x1, y1, c1 = append_all_lines(df, col_sub[7], lines_vac, time_centered, use_mask=1, mask=mask1)
x2, y2, c2 = append_all_lines(df, col_sub[7], lines_vac, time_centered, use_mask=1, mask=mask2)

In [ ]:
fs = 45
y_min = 10**-5
y_max = 10**6
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: $\eta$', fontsize=fs)
plt.scatter(x, y, c=c, alpha=0.5, s=10, cmap='plasma')
plt.vlines(505.5, y_min, y_max, color='C5', linewidth=5, label='Event 1', ls='--')
plt.vlines(545, y_min, y_max, color='C7', linewidth=5, label='Event 2', ls='--')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Unitless (%)', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.yscale('log')
plt.ylim(y_min, y_max)
plt.legend(fontsize=fs)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

In [ ]:
y_0 = np.log10(np.sort(y.copy()[y > 0]))
per = np.arange(y_0.shape[0]) / y_0.shape[0]
fig, axs = plt.subplots(1,2,figsize=[30,10])
axs[0].grid()
axs[0].plot(y_0, per)
axs[0].set_xlabel('Error')
axs[0].set_ylabel('% of Data < Error')
axs[1].grid()
axs[1].hist(y_0, bins=200)
axs[1].set_yscale('log')

In [ ]:
fs = 45
y_min = 10**-5
y_max = 10**6
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: $\eta$ (Event 1)', fontsize=fs)
plt.scatter(x1, y1, c=c1, alpha=0.5, s=1, cmap='plasma')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Unitless (%)', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.yscale('log')
plt.ylim(y_min, y_max)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

In [ ]:
fs = 45
y_min = 10**-5
y_max = 10**6
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: $\eta$ (Event 2)', fontsize=fs)
plt.scatter(x2, y2, c=c2, alpha=0.5, s=1, cmap='plasma')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Unitless (%)', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.yscale('log')
plt.ylim(y_min, y_max)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

## MSE

In [ ]:
x, y, c = append_all_lines(df, col_sub[8], lines_vac, time_centered, use_mask=0, mask=0)
x1, y1, c1 = append_all_lines(df, col_sub[8], lines_vac, time_centered, use_mask=1, mask=mask1)
x2, y2, c2 = append_all_lines(df, col_sub[8], lines_vac, time_centered, use_mask=1, mask=mask2)

In [ ]:
fs = 45
y_min = 10**-8
y_max = 10**4
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: MSE', fontsize=fs)
plt.scatter(x, y, c=c, alpha=0.5, s=10, cmap='plasma')
plt.vlines(505.5, y_min, y_max, color='C5', linewidth=5, label='Event 1', ls='--')
plt.vlines(545, y_min, y_max, color='C7', linewidth=5, label='Event 2', ls='--')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Relative Flux$^2$', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.yscale('log')
plt.ylim(y_min, y_max)
plt.legend(fontsize=fs)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

In [ ]:
fs = 45
y_min = 10**-8
y_max = 10**4
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: MSE (Event 1)', fontsize=fs)
plt.scatter(x1, y1, c=c1, alpha=0.5, s=1, cmap='plasma')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Relative Flux$^2$', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.yscale('log')
plt.ylim(y_min, y_max)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

In [ ]:
fs = 45
y_min = 10**-8
y_max = 10**4
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: MSE (Event 2)', fontsize=fs)
plt.scatter(x2, y2, c=c2, alpha=0.5, s=1, cmap='plasma')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Relative Flux$^2$', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.yscale('log')
plt.ylim(y_min, y_max)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

## Pearson

In [ ]:
x, y, c = append_all_lines(df, col_sub[9], lines_vac, time_centered, use_mask=0, mask=0)
x1, y1, c1 = append_all_lines(df, col_sub[9], lines_vac, time_centered, use_mask=1, mask=mask1)
x2, y2, c2 = append_all_lines(df, col_sub[9], lines_vac, time_centered, use_mask=1, mask=mask2)

In [ ]:
fs = 45
y_min = -0.6
y_max = 1.1
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: $R_{Pearson}$', fontsize=fs)
plt.scatter(x, y, c=c, alpha=0.5, s=10, cmap='plasma')
plt.vlines(505.5, y_min, y_max, color='C5', linewidth=5, label='Event 1', ls='--')
plt.vlines(545, y_min, y_max, color='C7', linewidth=5, label='Event 2', ls='--')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Unitless', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.ylim(y_min, y_max)
plt.legend(fontsize=fs)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

In [ ]:
y_0 = np.sort(y.copy()[y > 0])
per = 1 - (np.arange(y_0.shape[0]) / y_0.shape[0])
fig, axs = plt.subplots(1,2,figsize=[30,10])
axs[0].grid()
axs[0].plot(y_0, per)
axs[0].set_xlabel('Pearson')
axs[0].set_ylabel('% of Data > Pearson')
axs[1].grid()
axs[1].hist(y_0, bins=200)
axs[1].set_yscale('log')

In [ ]:
fs = 45
y_min = -0.6
y_max = 1.1
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: $R_{Pearson}$ (Event 1)', fontsize=fs)
plt.scatter(x1, y1, c=c1, alpha=0.5, s=1, cmap='plasma')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Unitless', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.ylim(y_min, y_max)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

In [ ]:
fs = 45
y_min = -0.6
y_max = 1.1
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: $R_{Pearson}$ (Event 2)', fontsize=fs)
plt.scatter(x2, y2, c=c2, alpha=0.5, s=1, cmap='plasma')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Unitless', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.ylim(y_min, y_max)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

## Spearman

In [ ]:
x, y, c = append_all_lines(df, col_sub[10], lines_vac, time_centered, use_mask=0, mask=0)
x1, y1, c1 = append_all_lines(df, col_sub[10], lines_vac, time_centered, use_mask=1, mask=mask1)
x2, y2, c2 = append_all_lines(df, col_sub[10], lines_vac, time_centered, use_mask=1, mask=mask2)

In [ ]:
fs = 45
y_min = -0.75
y_max = 1.1
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: $R_{Spearman}$', fontsize=fs)
plt.scatter(x, y, c=c, alpha=0.5, s=10, cmap='plasma')
plt.vlines(505.5, y_min, y_max, color='C5', linewidth=5, label='Event 1', ls='--')
plt.vlines(545, y_min, y_max, color='C7', linewidth=5, label='Event 2', ls='--')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Unitless', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.ylim(y_min, y_max)
plt.legend(fontsize=fs)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

In [ ]:
y_0 = np.sort(y.copy()[y > 0])
per = 1 - (np.arange(y_0.shape[0]) / y_0.shape[0])
fig, axs = plt.subplots(1,2,figsize=[30,10])
axs[0].grid()
axs[0].plot(y_0, per)
axs[0].set_xlabel('Spearman')
axs[0].set_ylabel('% of Data > Spearman')
axs[1].grid()
axs[1].hist(y_0, bins=200)
plt.show()

In [ ]:
fs = 45
y_min = -0.75
y_max = 1.1
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: $R_{Spearman}$ (Event 1)', fontsize=fs)
plt.scatter(x1, y1, c=c1, alpha=0.5, s=1, cmap='plasma')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Unitless', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.ylim(y_min, y_max)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

In [ ]:
fs = 45
y_min = -0.75
y_max = 1.1
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: $R_{Spearman}$ (Event 2)', fontsize=fs)
plt.scatter(x2, y2, c=c2, alpha=0.5, s=1, cmap='plasma')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Unitless', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.ylim(y_min, y_max)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

## pvalue

In [ ]:
x, y, c = append_all_lines(df, col_sub[11], lines_vac, time_centered, use_mask=0, mask=0)
x1, y1, c1 = append_all_lines(df, col_sub[11], lines_vac, time_centered, use_mask=1, mask=mask1)
x2, y2, c2 = append_all_lines(df, col_sub[11], lines_vac, time_centered, use_mask=1, mask=mask2)

In [ ]:
y_mod = y.copy()
y_mod[y_mod<10**-50] = 10**-50

In [ ]:
fs = 45
y_min = 10**-50
y_max = 1
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: p-value', fontsize=fs)
plt.scatter(x, y_mod, c=c, alpha=0.5, s=10, cmap='plasma')
plt.vlines(505.5, y_min, y_max, color='C5', linewidth=5, label='Event 1', ls='--')
plt.vlines(545, y_min, y_max, color='C7', linewidth=5, label='Event 2', ls='--')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Unitless', fontsize=fs)
plt.tick_params(labelsize=fs)
#plt.ylim(y_min, y_max)
plt.yscale('log')
plt.legend(fontsize=fs)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

In [ ]:
y_mod_ = y.copy()
y_mod_[y_mod_<10**-30] = 10**-30
y_0 = np.log10(np.sort(y_mod_.copy()[y_mod_ > 0]))
per = np.arange(y_0.shape[0]) / y_0.shape[0]
fig, axs = plt.subplots(1,2,figsize=[30,10])
axs[0].grid()
axs[0].plot(y_0, per)
axs[0].set_xlabel('p-value')
axs[0].set_ylabel('% of Data < p-value')
axs[1].grid()
axs[1].hist(y_0, bins=200)
axs[1].set_yscale('log')

In [ ]:
y1_mod = y1.copy()
y1_mod[y1_mod<10**-50] = 10**-50

In [ ]:
fs = 45
y_min = 10**-50
y_max = 1
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: p-value (Event 1)', fontsize=fs)
plt.scatter(x1, y1_mod, c=c1, alpha=0.5, s=1, cmap='plasma')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Unitless', fontsize=fs)
plt.tick_params(labelsize=fs)
#plt.ylim(y_min, y_max)
plt.yscale('log')
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

In [ ]:
y2_mod = y2.copy()
y2_mod[y2_mod<10**-50] = 10**-50

In [ ]:
fs = 45
y_min = 10**-50
y_max = 1
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: p-value (Event 2)', fontsize=fs)
plt.scatter(x2, y2_mod, c=c2, alpha=0.5, s=1, cmap='plasma')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Unitless', fontsize=fs)
plt.tick_params(labelsize=fs)
#plt.ylim(y_min, y_max)
plt.yscale('log')
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

## Slope

In [ ]:
x, y, c = append_all_lines(df, col_sub[12], lines_vac, time_centered, use_mask=0, mask=0)
x1, y1, c1 = append_all_lines(df, col_sub[12], lines_vac, time_centered, use_mask=1, mask=mask1)
x2, y2, c2 = append_all_lines(df, col_sub[12], lines_vac, time_centered, use_mask=1, mask=mask2)

In [ ]:
fs = 45
y_min = -10**2.5
y_max = 10**2.5
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: $m$', fontsize=fs)
plt.scatter(x, y, c=c, alpha=0.5, s=10, cmap='plasma')
plt.vlines(505.5, y_min, y_max, color='C5', linewidth=5, label='Event 1', ls='--')
plt.vlines(545, y_min, y_max, color='C7', linewidth=5, label='Event 2', ls='--')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Relative Flux / $\mathring{A}$', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.ylim(y_min, y_max)
plt.yscale('symlog')
plt.legend(fontsize=fs)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

In [ ]:
fs = 45
y_min = -10**2.5
y_max = 10**2.5
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: $m$ (Event 1)', fontsize=fs)
plt.scatter(x1, y1, c=c1, alpha=0.5, s=1, cmap='plasma')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Relative Flux / $\mathring{A}$', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.ylim(y_min, y_max)
plt.yscale('symlog')
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

In [ ]:
fs = 45
y_min = -10**2.5
y_max = 10**2.5
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: $m$ (Event 2)', fontsize=fs)
plt.scatter(x2, y2, c=c2, alpha=0.5, s=1, cmap='plasma')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Relative Flux / $\mathring{A}$', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.ylim(y_min, y_max)
plt.yscale('symlog')
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

## Intercept

In [ ]:
x, y, c = append_all_lines(df, col_sub[13], lines_vac, time_centered, use_mask=0, mask=0)
x1, y1, c1 = append_all_lines(df, col_sub[13], lines_vac, time_centered, use_mask=1, mask=mask1)
x2, y2, c2 = append_all_lines(df, col_sub[13], lines_vac, time_centered, use_mask=1, mask=mask2)

In [ ]:
fs = 45
y_min = -10**2.75
y_max = 10**2.75
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: $b$', fontsize=fs)
plt.scatter(x, y/10000, c=c, alpha=0.5, s=10, cmap='plasma')
plt.vlines(505.5, y_min, y_max, color='C5', linewidth=5, label='Event 1', ls='--')
plt.vlines(545, y_min, y_max, color='C7', linewidth=5, label='Event 2', ls='--')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Wavelength ($\mu$m)', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.ylim(y_min, y_max)
plt.yscale('symlog')
plt.legend(fontsize=fs)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

In [ ]:
fs = 45
y_min = -10**2.75
y_max = 10**2.75
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: $b$ (Event 1)', fontsize=fs)
plt.scatter(x1, y1/10000, c=c1, alpha=0.5, s=1, cmap='plasma')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Wavelength ($\mu$m)', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.ylim(y_min, y_max)
plt.yscale('symlog')
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

In [ ]:
fs = 45
y_min = -10**2.75
y_max = 10**2.75
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: $b$ (Event 2)', fontsize=fs)
plt.scatter(x2, y2/10000, c=c2, alpha=0.5, s=1, cmap='plasma')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Wavelength ($\mu$m)', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.ylim(y_min, y_max)
plt.yscale('symlog')
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

## Peak Ratio

In [ ]:
x, y, c = append_all_lines(df, col_sub[18]+'_ratio', lines_vac, time_centered, use_mask=0, mask=0)
x1, y1, c1 = append_all_lines(df, col_sub[18]+'_ratio', lines_vac, time_centered, use_mask=1, mask=mask1)
x2, y2, c2 = append_all_lines(df, col_sub[18]+'_ratio', lines_vac, time_centered, use_mask=1, mask=mask2)

In [ ]:
fs = 30
y_min = 10**-5
y_max = 10**7
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: Peak Ratio', fontsize=fs)
plt.scatter(x, y, c=c, alpha=0.5, s=10, cmap='plasma')
plt.vlines(505.5, y_min, y_max, color='C5', linewidth=5, label='Event 1', ls='--')
plt.vlines(545, y_min, y_max, color='C7', linewidth=5, label='Event 2', ls='--')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Unitless', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.ylim(y_min, y_max)
plt.yscale('log')
plt.legend(fontsize=fs)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

In [ ]:
fs = 30
y_min = 10**-5
y_max = 10**7
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: Peak Ratio (Event 1)', fontsize=fs)
plt.scatter(x1, y1, c=c1, alpha=0.5, s=1, cmap='plasma')
plt.hlines(1, 504, 507, ls='--', color='C9')
plt.hlines(10, 504, 507, ls='--', color='C0')
plt.hlines(0.1, 504, 507, ls='--', color='C0')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Unitless', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.ylim(y_min, y_max)
plt.yscale('log')
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)

In [ ]:
fs = 30
y_min = 10**-5
y_max = 10**7
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: Peak Ratio (Event 2)', fontsize=fs)
plt.scatter(x2, y2, c=c2, alpha=0.5, s=1, cmap='plasma')
plt.hlines(1, 543, 547, ls='--', color='C9')
plt.hlines(10, 543, 547, ls='--', color='C0')
plt.hlines(0.1, 543, 547, ls='--', color='C0')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Unitless', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.ylim(y_min, y_max)
plt.yscale('log')
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=fs)